In [1]:
from typing import List, Optional, Union, Tuple
from abstractions.goap.actions import Prerequisites, Goal
from abstractions.goap.entity import Statement, Attribute
from abstractions.goap.nodes import GameEntity, Node, Position
from abstractions.goap.language_state import GoalState
from abstractions.goap.gridmap import GridMap
from abstractions.goap.interactions import Character, Door, Key, Treasure, Floor, Wall, InanimateEntity, IsPickupable, TestItem, Open, Close, Unlock, Lock, Pickup, Drop, Move
from abstractions.goap.game.main import generate_dungeon
from abstractions.goap.shapes import Rectangle, Shadow, Radius, RayCast, BlockedRaycast
from abstractions.goap.agents import Agent

def source_target_position_comparison(source: tuple[int,int], target: tuple[int,int]) -> bool:
    """Check if the source entity's position is the same as the target entity's position."""
    if source and target:
        return source == target
    return False

def treasure_in_neighborhood(source: GameEntity, target: Optional[GameEntity] = None) -> bool:
    """Check if the treasure is in the character's neighborhood."""
    if source.node and target and target.node:
        return source.node in target.node.neighbors()
    return False

def key_in_inventory(source: GameEntity, target: Optional[GameEntity] = None) -> bool:
    """Check if the key is in the character's inventory."""
    if target:
        return target in source.inventory
    return False

def is_treasure(source: GameEntity, target: Optional[GameEntity] = None) -> bool:
    """Check if the target entity is a Treasure."""
    return isinstance(target, Treasure)

def is_golden_key(source: GameEntity, target: Optional[GameEntity] = None) -> bool:
    """Check if the entity is a Golden Key."""
    return isinstance(target, Key) and target.key_name.value == "Golden Key"

def is_door(entity: GameEntity, target: Optional[GameEntity] = None) -> bool:
    """Check if the entity is a Door."""
    return isinstance(entity, Door)

# Example usage
grid_map = GridMap(width=10, height=10)
grid_map.register_actions([Move, Pickup,  Open, Close, Unlock, Lock])
room_width, room_height = 6, 6
character, door, key, treasure = generate_dungeon(grid_map, room_width, room_height)
grid_map.generate_entity_type_map()

# Goal 1: Check if the character's position is the same as the treasure's position
reach_treasure_goal = Goal(
    name="Reach the treasure",
    source_entity_id=character.id,
    target_entity_id=treasure.id,
    prerequisites=Prerequisites(
        source_statements=[Statement(conditions={"can_act": True})],
        target_statements=[Statement(callables=[is_treasure])],
        source_target_statements=[
            Statement(comparisons={
                "source_position": ("position", "position", source_target_position_comparison)
            })
        ]
    )
)

# Goal 2: Check if the treasure is in the character's neighborhood
treasure_in_neighborhood_goal = Goal(
    name="Treasure in neighborhood",
    source_entity_id=character.id,
    target_entity_id=treasure.id,
    prerequisites=Prerequisites(
        source_statements=[Statement(conditions={"can_act": True})],
        target_statements=[Statement(callables=[is_treasure])],
        source_target_statements=[
            Statement(callables=[treasure_in_neighborhood])
        ]
    )
)

# Goal 3: Check if the key is in the character's inventory
key_in_inventory_goal = Goal(
    name="Key in inventory",
    source_entity_id=character.id,
    target_entity_id=key.id,
    prerequisites=Prerequisites(
        source_statements=[Statement(conditions={"can_act": True})],
        target_statements=[Statement(callables=[is_golden_key])],
        source_target_statements=[
            Statement(callables=[key_in_inventory])
        ]
    )
)

# Goal 4: Check if the door is unlocked
door_unlocked_goal = Goal(
    name="Door unlocked",
    source_entity_id=door.id,
    prerequisites=Prerequisites(
        source_statements=[
            Statement(callables=[is_door]),
            Statement(conditions={"is_locked": False})
        ],
        target_statements=[],
        source_target_statements=[]
    )
)

goals = [reach_treasure_goal, treasure_in_neighborhood_goal, key_in_inventory_goal, door_unlocked_goal]
# goals = [key_in_inventory_goal]
model_path = "/Users/tommasofurlanello/Documents/Dev/models/TheBloke/dolphin-2.5-mixtral-8x7b-GGUF/dolphin-2.5-mixtral-8x7b.Q8_0.gguf"
# model_path = "/Users/tommasofurlanello/Documents/Dev/models/NousResearch/Nous-Hermes-2-Mixtral-8x7B-DPO-GGUF/Nous-Hermes-2-Mixtral-8x7B-DPO.Q8_0.gguf"
# model_path = "/Users/tommasofurlanello/Documents/Dev/models/NousResearch/Hermes-2-Pro-Mistral-7B-GGUF/Hermes-2-Pro-Mistral-7B.Q8_0.gguf"
model_path = None
agent = Agent(
    goals=goals,
    character_id=character.id,
    actions=grid_map.get_actions(),
    entity_type_map=grid_map.entity_type_map,
    llm = model_path

)
mdp = False
if mdp:
    shape = grid_map.get_rectangle()
else:
    character_node = GameEntity.get_instance(character.id).node
    shape = grid_map.get_shadow(source=character_node, max_radius=5)



pygame-ce 2.4.1 (SDL 2.28.5, Python 3.10.6)
Updating block attributes... for door


In [2]:
agent.derive_allowed_strings(grid_map)

['East Move Floor',
 'North Move Floor',
 'NorthEast Move Floor',
 'NorthWest Move Floor',
 'West Move Floor']

In [3]:
agent.allowed_strings_count

{}

In [4]:
agent.run(grid_map, max_steps=10000, mdp=True)


--- Step 0 ---

Action State:


Generated Action:
East Move Floor
Attemping to find entity of type <class 'abstractions.goap.interactions.Floor'> in node name='' id='240667f6-bb89-4899-9346-d2d21d46b3d1' position=Position(name='Position', id='e622b957-6858-466b-a818-13f05bb476ac', value=(6, 1)) entities=[Floor(id=a0e698f1-57bd-4bdd-81eb-85cc1a6568ef, name=Floor_6_1, position=(6, 1))] gridmap_id='4181b692-f2c5-43bd-ab33-7bed1b8bc28e' blocks_movement=BlocksMovement(name='BlocksMovement', id='089bf88f-9adc-4e23-886d-37fc945ff274', value=False) blocks_light=BlocksLight(name='BlocksLight', id='b5326e0c-c4a1-4433-92c9-134e03abd8a9', value=False) hash_resolution='default' starting from action string East Move Floor

Action Payload:
actions=[ActionInstance(source_id='4a30dc8d-a8be-4139-97ad-1ab0579189de', target_id='a0e698f1-57bd-4bdd-81eb-85cc1a6568ef', action=Move(name='Move Step', prerequisites=Prerequisites(source_statements=[Statement(name='Statement', id='27bfe83d-e54f-41ed-9acd-a8c81e2

In [5]:
# obs_text, act_text, goal_text = agent.generate_text(shape)

In [6]:
# agent.generate_action(obs_text, act_text, goal_text)

In [7]:
# agent = Agent(
#     goals=goals,
#     character_id=character.id,
#     actions=grid_map.get_actions(),
#     entity_type_map=grid_map.entity_type_map
# )

# # Run the agent
# agent.run(grid_map, max_steps=10000, mdp=True)

In [8]:
# gang = Character.get_instance(character.id)
